In [11]:
import pandas as pd
import quandl #http://127.0.0.1:8888/?token=09794f0b843849dac8d164691e7fb4f9c2818455909b9f6e
#token
import numpy as np
from sklearn import preprocessing,cross_validation, svm
from sklearn.linear_model import LinearRegression
#here we are using slearn for scaling the data 
# scaling is usually done on features to keep it between 
#-1 to 1 it helps in

C:\Users\Subham\Anaconda2\envs\keras_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
df= quandl.get('WIKI/GOOGL')

In [13]:
print(df.head())
# to check the features
# try to think about relationships between algorithms

              Open    High     Low    Close      Volume  Ex-Dividend  \
Date                                                                   
2004-08-19  100.01  104.06   95.96  100.335  44659000.0          0.0   
2004-08-20  101.01  109.08  100.50  108.310  22834300.0          0.0   
2004-08-23  110.76  113.48  109.05  109.400  18256100.0          0.0   
2004-08-24  111.24  111.60  103.57  104.870  15247300.0          0.0   
2004-08-25  104.76  108.00  103.88  106.000   9188600.0          0.0   

            Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  \
Date                                                                   
2004-08-19          1.0  50.159839  52.191109  48.128568   50.322842   
2004-08-20          1.0  50.661387  54.708881  50.405597   54.322689   
2004-08-23          1.0  55.551482  56.915693  54.693835   54.869377   
2004-08-24          1.0  55.792225  55.972783  51.945350   52.597363   
2004-08-25          1.0  52.542193  54.167209  52.100830   53.1

In [14]:
# garbbing features
df = df [['Adj. Open','Adj. High','Adj. Low','Adj. Close','Adj. Volume']]
df['HL_PCT']= (df['Adj. High']-df['Adj. Close'])/ df['Adj. Close']*100.0
df['PCT_change']= (df['Adj. Close']-df['Adj. Open'])/ df['Adj. Open']*100.0

df=df[['Adj. Close','PCT_change','HL_PCT','Adj. Volume']]
print (df.head())

            Adj. Close  PCT_change    HL_PCT  Adj. Volume
Date                                                     
2004-08-19   50.322842    0.324968  3.712563   44659000.0
2004-08-20   54.322689    7.227007  0.710922   22834300.0
2004-08-23   54.869377   -1.227880  3.729433   18256100.0
2004-08-24   52.597363   -5.726357  6.417469   15247300.0
2004-08-25   53.164113    1.183658  1.886792    9188600.0


In [15]:
import math
forecast_col = 'Adj. Close'
#just in case there is missing data
df.fillna(-99999, inplace=True)
#in machine learning we cannot work with Nan so we give
#it a outliner value or remove the data 
# and since we  alredy dont have enough data in the
# world we cannot afford to lose more data

forecast_out=int(math.ceil(0.01*len(df)))# no of days out
# tring to prdict out 10% of the size of your data frame
# math.ceil returns float and we dont want that

############# we have features  noe we need lables####

df['label']= df[forecast_col].shift(-forecast_out)
#so what we did here was that in order to create the lable 
#lables we need to understand what lables are 
# lables are information for the algorithm which says 
#for this perticular input value the predicted value 
# is that label
df.dropna(inplace=True)
print (df.head())

            Adj. Close  PCT_change    HL_PCT  Adj. Volume      label
Date                                                                
2004-08-19   50.322842    0.324968  3.712563   44659000.0  68.752232
2004-08-20   54.322689    7.227007  0.710922   22834300.0  69.639972
2004-08-23   54.869377   -1.227880  3.729433   18256100.0  69.078238
2004-08-24   52.597363   -5.726357  6.417469   15247300.0  67.839414
2004-08-25   53.164113    1.183658  1.886792    9188600.0  68.912727


In [16]:
# generally feaatures nad lables are dinoted by x and y
# X = np.array(df.drop(['label'],1))
# y = np.array(df['label'])
# scale x
# X = preprocessing.scale(X)
# X - features
# y- labels
# why is scaling time consuming ?
# now that we have scaled our features and passed it to a classifier and we have now trainedour classifier ,
#  now that new dat enters the system while testing 
# we need to make sure we scale the new data too before sending it into the clssifier but the 
# problem here is that we wil have to scale the new datt 
# we all the training dat to get the correct scaling

# X = X[:-forecast_out+1]# we are making sure that we use X's for which we have values in yy
# as you might remember we had shifted the Adj. Close by forecats out
# df.dropna(inplace=True)
# y= np.array(df['label'])
# print (len(X),len(y))

3178 3178


In [28]:
# X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)
# clf = LinearRegression(n_jobs=-1)#no of treads that linear regresion supports for traing
# clf.fit(X_train,y_train)
# accuracy=clf.score(X_test,y_test)
# print (accuracy)

0.972837852519


In [38]:
############regresion forecatsing#############
X = np.array(df.drop(['label'],1))
X = preprocessing.scale(X)
X = X[:-forecast_out]
X_lately = X[-forecast_out:]
df.dropna(inplace=True)
y= np.array(df['label'])
y= y[forecast_out:]
print(len(X),len(y))

3145 3145


In [39]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)
clf = LinearRegression(n_jobs=-1)#no of treads that linear regresion supports for traing
clf.fit(X_train,y_train)
accuracy=clf.score(X_test,y_test)
print (accuracy)

0.94572260801


In [40]:
forecast_set= clf.predict(X_lately)
print (forecast_set, accuracy,forecast_out)

[ 828.62436306  827.1107164   816.06314423  831.55883706  831.70057668
  837.17370418  848.75241069  851.38986825  850.36730932  854.27546079
  853.94288709  855.23532463  851.80227761  853.6575883   848.52968876
  852.64695259  868.00486323  873.80498082  882.84318543  880.41998055
  867.39987432  846.70506582  844.069087    838.86087456  841.96379663
  844.15750101  845.99244525  853.2700277   854.13447449  854.57281877
  859.1469348   863.3369438   864.46886843] 0.94572260801 33
